In [2]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time


from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

from scipy import stats
from matplotlib import pyplot
from scipy.stats import spearmanr

import warnings
warnings.filterwarnings("ignore")


import matplotlib
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')

import matplotlib.font_manager as fm


import seaborn as sns

table = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v4/A5.pkl")
iHiVred = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v4/iHiV4.pkl")
iHiV = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v4/iHiV5.pkl")
VHred = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v4/VH4.pkl")
tableg = table.T

iFT = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v4/iFT4.pkl")

VH = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v4/VH5.pkl")
tableg = table.T
B = pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v4/B5.pkl")

Bdf = pd.DataFrame(B)

species_ones = [] 
for i in range(5):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(5):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

sp_list_provided = ['LP', 'LB', 'AP', 'AT', 'AO']

#### get all data -- design matrix, steady state abundances, and list of wells present

sst = pd.read_pickle("~/compressed_sensingv1/realdatasets/BENCHMARKING/Will-Luddington-v1/5sp-Gould-CFU-data.pkl")
sstv3 = sst.copy()

from sklearn.metrics import r2_score


import matplotlib.patches as mpatches

from sklearn.ensemble import RandomForestRegressor


from sklearn.metrics import mean_absolute_percentage_error as mape



In [3]:
csdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/BENCHMARKING/Will-Luddington-v1/CFU_CS_WH.pkl")

In [4]:
rdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/BENCHMARKING/RANDOM FOREST/RF-5sp-will-CFU.pkl")

In [5]:
cv_splitsl = [3] 
max_depthl = [2,5,10,15]
ntrees = [100,200,300,400,500,1000]
max_featuresl = ["auto","sqrt","log2"]

In [6]:
samp = rdf[rdf["k-fold"]==3]
samp.head(2)

,species,max_depth,n_trees,Xtrain,Xtest,ypred,k-fold,max_features
0,0,2,100,"[9, 3, 6, 12, 7, 1, 0, 4, 10, 15]","[14, 13, 2, 5, 8, 11]","[228340.73472638876, 237717.67395833324, 27672...",3,auto
1,0,2,100,"[14, 13, 2, 5, 8, 11, 1, 0, 4, 10, 15]","[9, 3, 6, 12, 7]","[241571.46132569443, 261492.07496736123, 25694...",3,auto


In [7]:
crossval = [] 
for m in cv_splitsl:
    samp = rdf[rdf["k-fold"]==m]
    for species in range(5):
        abunvec = pd.DataFrame(sstv3[species][species_ones[species]].values)
        sp1 = samp[samp["species"]==species]
        for md in max_depthl:
            rho1 = sp1[sp1["max_depth"]==md]
            for nest in ntrees: 
                ntre = rho1[rho1["n_trees"]==nest]
                for mfeats in max_featuresl:
                    mdf = ntre[ntre["max_features"]==mfeats]
                    r2 = [] 
                    yallbest  = []
                    yallact  = [] 
                    for iterii in range(m):
                        idx = mdf["Xtest"].iloc[iterii]
                        yact = abunvec.T[idx].T[0].values.astype(float)
                        ybest = mdf["ypred"].iloc[iterii]
                        ybestdf1 = pd.DataFrame(ybest)
                        ybestdf1[ybestdf1[0]<1e-10]=0
                        slope, intercept,r_value, p_value, std_err = stats.linregress(yact,ybestdf1[0].values)
                        yallbest.append(ybestdf1[0].values)
                        yallact.append(yact)
                        rval = r2_score(yact,ybestdf1[0].values)
                        gop = 1/(2-rval)
                        mape1 = mape(yact,ybestdf1[0].values)
                        r2.append([gop,rval,mape1,slope,intercept,r_value**2])

                    r2df = pd.DataFrame(r2)
                    if m == 2:
                        ybestagg = [*yallbest[0],*yallbest[1]]
                        yactagg = [*yallact[0],*yallact[1]]
                    if m == 3:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2]]
                    if m == 5:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4]]
                    if m == 7:
                        ybestagg = [*yallbest[0],*yallbest[1],*yallbest[2],*yallbest[3],*yallbest[4],*yallbest[5],*yallbest[6]]
                        yactagg = [*yallact[0],*yallact[1],*yallact[2],*yallact[3],*yallact[4],*yallact[5],*yallact[6]]
                    #print(r2_score(yactagg,ybestagg))
                    #print(yactagg)
                    slope, intercept,r_value, p_value, std_err = stats.linregress(yactagg,ybestagg)
                    rval2 = r2_score(yactagg,ybestagg)
                    gop2 = 1/(2-rval2)
                    mean1 = r2df.mean().values[0]
                    mean2 = r2df.mean().values[1]
                    mean3 = r2df.mean().values[2]
                    mean4 = r2df.mean().values[3]
                    mean5 = r2df.mean().values[4]
                    mean6 = r2df.mean().values[5]
                    mape2 = mape(yactagg,ybestagg)
                    crossval.append([m,species,md,nest,mfeats,gop2,rval2,mape2,slope,intercept,r_value**2,"break",mean1,mean2,mean3,mean4,mean5,mean6,yactagg,ybestagg])

In [8]:
crossdf = pd.DataFrame(crossval,columns=["k-fold","sp","max_depth","trees","features","nnseagg","r2agg","mapeagg","slagg","intagg","rlinagg","means on right","nnseavg","r2avg","mapeavg","slavg","intavg","rlinavg","testact","testpred"])

In [9]:
crossdf.head(4)

,k-fold,sp,max_depth,trees,features,nnseagg,r2agg,mapeagg,slagg,intagg,rlinagg,means on right,nnseavg,r2avg,mapeavg,slavg,intavg,rlinavg,testact,testpred
0,3,0,2,100,auto,0.488074,-0.048870,0.095814,0.145795,216001.688920,0.062561,break,0.440673,-0.316097,0.096632,0.227194,195833.775209,0.118566,"[239860.64583333334, 250276.77083333334, 24321...","[228340.73472638876, 237717.67395833324, 27672..."
1,3,0,2,100,sqrt,0.505003,0.019814,0.086857,0.078504,232230.299203,0.045871,break,0.481776,-0.080834,0.088089,0.153195,213889.738794,0.149444,"[239860.64583333334, 250276.77083333334, 24321...","[235309.66390277774, 239088.9742743055, 267920..."
2,3,0,2,100,log2,0.520515,0.078827,0.084616,0.104927,226547.550909,0.084306,break,0.496957,-0.024577,0.085906,0.195641,204136.067635,0.218237,"[239860.64583333334, 250276.77083333334, 24321...","[234283.36366894824, 245567.84124513887, 26543..."
3,3,0,2,200,auto,0.501046,0.004177,0.092054,0.172239,209737.711144,0.087214,break,0.450933,-0.278136,0.092708,0.274824,184118.061315,0.207768,"[239860.64583333334, 250276.77083333334, 24321...","[232223.3338812501, 240388.43856041666, 272027..."
